<a href="https://colab.research.google.com/github/Tomawock/NLP_Attack/blob/main/models/studio_1/DDI.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [1]:
import pickle
import pandas as pd
import optuna
import numpy as np
from sklearn.preprocessing import OneHotEncoder
from tensorflow import keras
from sklearn.utils.class_weight import compute_class_weight

In [ ]:
path="/content/drive/Shareddrives/Deep Learning/datasets/DDI/"

In [ ]:
dev = pd.read_csv(path+'\ddi2013-type\\dev.tsv', sep='\t')
test = pd.read_csv(path+'\ddi2013-type\\test.tsv', sep='\t')
train = pd.read_csv(path+'\ddi2013-type\\train.tsv', sep='\t')

data_sinonimi = pd.read_csv(path+"\ddi2013-type\\DDI_sinonimi_test.csv")
data_embedding = pd.read_csv(path+"\ddi2013-type\\DDI_embedding_test.csv")

data_sinonimi_train = pd.read_csv(path+"\ddi2013-type\\DDI_sinonimi.csv")

print(test.shape)
print(data_sinonimi.shape)
print(test)

In [ ]:
test_study_1 = pd.concat([test, data_sinonimi], ignore_index=True)
train_study_1 = pd.concat([train, data_sinonimi_train], ignore_index=True)
print (test_study_1.shape)
print (train_study_1.shape)

In [ ]:
print(train.shape)
print(data_sinonimi.shape)
print(data_embedding.shape)

In [ ]:
print(train.label.value_counts())
print(data_sinonimi.label.value_counts())
print(data_embedding.label.value_counts())

In [6]:
with open(path+"\word2index.pkl", 'rb') as output:
  w2i = pickle.load(output)
with open(path+"\embedding_matrix.pkl", 'rb') as output:
  embedding_matrix = pickle.load(output)

Set up per allenamento Modello

In [8]:
categories = [['DDI-false', 'DDI-mechanism', 'DDI-effect', 'DDI-advise','DDI-int']]

my_text_to_word_sequence = lambda sen: keras.preprocessing.text.text_to_word_sequence(sen,
                                                                                      filters='!"#&()*+,-./:;<=>?[\\]^_`\'{|}~\t\n',
                                                                                      lower=True)

TRAINSET

In [88]:
five_hot_train = OneHotEncoder(sparse=False, categories=categories).fit_transform(
  train_study_1.label.to_numpy().reshape(-1, 1)
)

In [89]:
sentences_train = [my_text_to_word_sequence(sentence) for sentence in train_study_1['sentence']]

TESTSET

In [90]:
five_hot_test = OneHotEncoder(sparse=False, categories=categories).fit_transform(
  test_study_1.label.to_numpy().reshape(-1, 1)
)

In [91]:
sentences_test = [my_text_to_word_sequence(sentence) for sentence in test_study_1['sentence']]

ORIGINALE

In [92]:
five_hot_orig = OneHotEncoder(sparse=False, categories=categories).fit_transform(
  test.label.to_numpy().reshape(-1, 1)
)

In [93]:
sentences_orig = [my_text_to_word_sequence(sentence) for sentence in test['sentence']]

SINONIMI

In [94]:
five_hot_sin = OneHotEncoder(sparse=False, categories=categories).fit_transform(
  data_sinonimi.label.to_numpy().reshape(-1, 1)
)

In [95]:
sentences_sin = [my_text_to_word_sequence(sentence) for sentence in data_sinonimi['sentence']]

EMBEDDING

In [96]:
five_hot_emb = OneHotEncoder(sparse=False, categories=categories).fit_transform(
  data_embedding.label.to_numpy().reshape(-1, 1)
)

In [97]:
sentences_emb = [my_text_to_word_sequence(sentence) for sentence in data_embedding['sentence']]

Estrai la massima dimensione dell'input in base ai vari dataset considerati

In [67]:
max_index, max = (-1, -1)
for i, sentence in enumerate(sentences_train):
  max_index, max = (i, len(sentence)) if len(sentence) > max else (max_index, max)
for i, sentence in enumerate(sentences_test):
  max_index, max = (i, len(sentence)) if len(sentence) > max else (max_index, max)
for i, sentence in enumerate(sentences_orig):
  max_index, max = (i, len(sentence)) if len(sentence) > max else (max_index, max)
for i, sentence in enumerate(sentences_sin):
  max_index, max = (i, len(sentence)) if len(sentence) > max else (max_index, max)
for i, sentence in enumerate(sentences_emb):
  max_index, max = (i, len(sentence)) if len(sentence) > max else (max_index, max)
  
print(f'Il massimo è {max}')

Il massimo è 92


Crao i vari embedding per tutti i dataset, quest'operazione e pesante 

In [68]:
embedded_trainset = np.zeros(shape=(len(sentences_train), max, 300))
for i, sentence in enumerate(sentences_train):
  for j, word in enumerate(sentence):
    try:
      embedded_trainset[i, j, :] = embedding_matrix[w2i[word]]
    except KeyError:
      pass

In [76]:
embedded_origin = np.zeros(shape=(len(sentences_orig), max, 300))
for i, sentence in enumerate(sentences_orig):
  for j, word in enumerate(sentence):
    try:
      embedded_trainset[i, j, :] = embedding_matrix[w2i[word]]
    except KeyError:
      pass

In [77]:
embedded_testset = np.zeros(shape=(len(sentences_test), max, 300))
for i, sentence in enumerate(sentences_test):
  for j, word in enumerate(sentence):
    try:
      embedded_testset[i, j, :] = embedding_matrix[w2i[word]]
    except KeyError:
      pass

In [78]:
embedded_sin = np.zeros(shape=(len(sentences_sin), max, 300))
for i, sentence in enumerate(sentences_sin):
  for j, word in enumerate(sentence):
    try:
      embedded_sin[i, j, :] = embedding_matrix[w2i[word]]
    except KeyError:
      pass

In [79]:
embedded_emb = np.zeros(shape=(len(sentences_emb), max, 300))
for i, sentence in enumerate(sentences_emb):
  for j, word in enumerate(sentence):
    try:
      embedded_emb[i, j, :] = embedding_matrix[w2i[word]]
    except KeyError:
      pass

Carica optuna results e inizializza il modello, oppure salva il modello oppure carica solo i pesi del modello 

In [26]:
best_params = optuna.load_study(study_name="DDI", storage="sqlite:///"+path+"\ddi2013-type\\optuna_ddi_studio_0.db").best_params

In [ ]:
print(f'{best_params}')

In [28]:
model = keras.Sequential()
model.add(keras.layers.Input(shape=(max, 300)))
model.add(keras.layers.Bidirectional(layer=keras.layers.LSTM(units=best_params['units'],
                                                             recurrent_dropout=best_params['dropout'],
                                                             activation='tanh')))

model.add(keras.layers.Dense(5, activation='softmax'))
model.compile(loss='categorical_crossentropy',
              optimizer=keras.optimizers.Adam(learning_rate=0.001),
              metrics=['accuracy'])

In [ ]:
#train the model
result = model.fit(embedded_trainset,
                   five_hot_train,
                   epochs=100,
                   batch_size=best_params['batch_size'],
                   callbacks=[keras.callbacks.EarlyStopping(monitor='loss',
                                                            patience=10)])

In [31]:
#save the model
model.save_weights(path+'\ddi2013-type\\DDI_w_studio_1.h5')

In [47]:
#load only the w of the model, the model must be already executed
model.load_weights(path+'\ddi2013-type\\DDI_w_studio_1.h5')

EVALUATE ALL DATASET

In [98]:
result_base=model.evaluate(embedded_trainset, five_hot_train, batch_size=best_params['batch_size'],)
print(f'DATASET ORIGINARIO{result_base}')

422/422 [==============================] - 25s 60ms/step - loss: 0.0957 - accuracy: 0.9675
DATASET ORIGINARIO[0.09572969377040863, 0.9674902558326721]


In [99]:
result_base=model.evaluate(embedded_testset, five_hot_test, batch_size=best_params['batch_size'],)
print(f'DATASET TEST{result_base}')

130/130 [==============================] - 8s 61ms/step - loss: 0.9669 - accuracy: 0.8144
DATASET TEST[0.9669227600097656, 0.8144419193267822]


In [100]:
result_base=model.evaluate(embedded_origin, five_hot_orig, batch_size=best_params['batch_size'],)
print(f'DATASET TEST{result_base}')

65/65 [==============================] - 6s 91ms/step - loss: 0.9027 - accuracy: 0.8301
DATASET TEST[0.9027319550514221, 0.8300642371177673]


In [101]:
result_base=model.evaluate(embedded_sin, five_hot_sin, batch_size=best_params['batch_size'],)
print(f'DATASET SINONIMI{result_base}')

65/65 [==============================] - 4s 61ms/step - loss: 1.0155 - accuracy: 0.8179
DATASET SINONIMI[1.0154950618743896, 0.8179135322570801]


In [102]:
result_base=model.evaluate(embedded_emb, five_hot_emb, batch_size=best_params['batch_size'],)
print(f'DATASET EMBEDDING{result_base}')

65/65 [==============================] - 4s 59ms/step - loss: 1.0251 - accuracy: 0.8118
DATASET EMBEDDING[1.0250608921051025, 0.8118382096290588]


tali test sono stati effettuati su windows, eventuali path necessitano di essere adattati, se effettuati su sistema operativo unix like.

